# Installing Library

In [1]:
# uncomment, install ini dulu
# ini ku uncomment soalnya udh nginstall,
# takut makan kuota wakakaka

# %pip install tensorflow
# %pip install tensorflow_hub
# %pip install scikit-learn
# %pip install tensorflow vggish-slim
# %pip install librosa
# %pip install resampy librosa tensorflow numpy


# Import Library

In [5]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
from tensorflow.keras import layers, models, optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from vggish import mel_features
from vggish import vggish_input, vggish_params, vggish_postprocess, mel_features
# import vggish_input
# import vggish_params
# import vggish_postprocess

ModuleNotFoundError: No module named 'mel_features'

Load VGGish model

In [ ]:
def load_vggish_model():
    vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
    return vggish_model

Preprocess the Audio

In [ ]:
def preprocess_audio_vggish(audio_path):
    
    audio_data, sample_rate = librosa.load(audio_path, sr=vggish_params.SAMPLE_RATE)

    examples_batch = vggish_input.waveform_to_examples(audio_data, sample_rate)
    
    return examples_batch

Load audio files and labels

In [ ]:
def load_audio_files(directory):
    audio_files = []
    labels = []
    
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            label_file = os.path.join(folder_path, 'new_label.txt')
            with open(label_file, 'r') as f:
                sds_score = float(f.read().strip())
                if sds_score < 40:
                    label = 'low'
                elif 41 >= sds_score <= 60:
                    label = 'medium'
                else:
                    label = 'high'
            for file in os.listdir(folder_path):
                if file.endswith('_out.wav'):
                    audio_path = os.path.join(folder_path, file)
                    audio_files.append(audio_path)
                    labels.append(label)
                    
    return audio_files, labels

Extracting features using VGGish

In [ ]:
def extract_vggish_features(vggish_model, audio_files):
    features = []
    for file in audio_files:
        examples_batch = preprocess_audio_vggish(file)
        embeddings = vggish_model(examples_batch)
        features.append(np.mean(embeddings, axis=0))
    return np.array(features)

Custom model

In [ ]:
# def build_model(input_shape):
#     model = tf.keras.Sequential([
#         tf.keras.layers.InputLayer(input_shape=input_shape),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(3, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model

In [ ]:
# Load VGGish model
vggish_model = load_vggish_model()

In [ ]:
inputs = tf.keras.Input(shape=(), dtype=tf.float32)
x = vggish_model(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation='softmax')(x)

TypeError: Binding inputs to tf.function failed due to `A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
`. Received args: (<KerasTensor shape=(None,), dtype=float32, sparse=None, name=keras_tensor>,) and kwargs: {} for signature: (waveform: TensorSpec(shape=(None,), dtype=tf.float32, name=None)).

In [ ]:
model = models.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Load data and model

Load data and model

In [ ]:
# Load data
train_directory = 'EATD-Corpus/train_data'
validation_directory = 'EATD-Corpus/validation_data'

Load and extract training data

In [ ]:
# Load training data
train_audio_files, train_labels = load_audio_files(train_directory)

# Extract features for training data
train_features = extract_vggish_features(vggish_model, train_audio_files)

# Encode labels
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_labels)

NameError: name 'vggish_params' is not defined

Load and extract validation data

In [ ]:
# Load validationation data
validation_audio_files, validation_labels = load_audio_files(validation_directory)

# Extract features for validationation data
validation_features = extract_vggish_features(vggish_model, validation_audio_files)

# Encode labels
validation_labels_encoded = le.transform(validation_labels)

# Custom Model

# Train Model

In [ ]:
EPOCH = 20

input_shape = train_features.shape[1:]
model = build_model(input_shape)
model.fit(train_features, train_labels_encoded, 
          epochs=EPOCH, 
          batch_size=32, 
          validation_data=(validation_features, validation_labels_encoded))

Evaluate Model

In [ ]:
validation_pred = model.predict(validation_features)
validation_pred_labels = np.argmax(validation_pred, axis=1)

Tes metrik evaluasi

In [ ]:
# Print evaluation metrics
print(classification_report(validation_labels_encoded, validation_pred_labels, target_names=le.classes_))
print(confusion_matrix(validation_labels_encoded, validation_pred_labels))